In [124]:
# This script can be run to create a dataset of google news headlines. 

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs4

In [125]:
%%time
searchTerm = "something"
count=0
url = ("https://www.google.com/search?q=" + searchTerm + "&hl=en&gl=us&authuser=0&tbs=qdr:y&tbm=nws&start=$count$&cad=h")

headlines = []
source_time = []

while (count<50):
    new_url = url.replace("$count$", str(count))
    r = requests.get(new_url)
    count = count+10 
    soup = bs4(r.text, 'html.parser')
    
    source_time += soup.find_all("div", "slp")
    
    for h in soup.find_all("div", "st"):
        headlines.append(h.text)

Wall time: 4.39 s


In [126]:
source_time_text  = []

def textify(): 
    for d in source_time:
        source_time_text.append(d.find('span').text)        
    return print(source_time_text)
# textify()
# len(source_time_text)

In [127]:
# print(headlines)
# len(headlines)